In [ ]:
##!/usr/bin/env python
"""plot_mme_vs_sme.py

This script compares the PDFs of the EHI values for the CMIP Multi-Model Ensembles (MME)
and the Single-Model Ensembles (SME) against Berkeley Earth for different regions

Using the Excess Heat Index 
Baseline Period: 1950-2014, 90th Percentile Threshold

Author: Annette L Hirsch @ CLEX, UNSW. Sydney (Australia)
email: a.hirsch@unsw.edu.au
Created: Wed Nov 11 10:54:39 AEDT 2020

"""

## Load Python Packages

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import netCDF4 as nc
import matplotlib.pyplot as plt
import xarray as xr
import xesmf as xe
import regionmask
import seaborn as sns; sns.set(style="white", color_codes=True)

### Updated IPCC regions

This is a python package containing the new regions: import regionmask

Iturbide, M., Guti�errez, J.M., Alves, L.M., Bedia, J., Cimadevilla, E., Cofi~no, A.S., Cerezo-Mota, R., Di Luca, A., Faria, S.H., Gorodetskaya, I., Hauser, M., Herrera, S., Hewitt, H.T., Hennessy, K.J., Jones, R.G., Krakovska, S., Manzanas, R., Marínez-Castro, D., Narisma, G.T., Nurhati, I.S., Pinto, I., Seneviratne, S.I., van den Hurk, B., Vera, C.S., 2020. An update of IPCC climate reference regions for subcontinental analysis of climate model data: definition and aggregated datasets. Earth Syst. Sci. Data Discuss. https://doi.org/10.5194/essd-2019-258

Git repo with the new regions: https://github.com/SantanderMetGroup/ATLAS/tree/master/reference-regions

In [ ]:
rlabels = ['GIC', 'NWN', 'NEN', 'WNA', 'CNA', 'ENA', 'NCA', 'SCA', 'CAR', 'NWS',
                  'NSA', 'NES', 'SAM', 'SWS', 'SES', 'SSA', 'NEU', 'WCE', 'EEU', 'MED',
                  'SAH', 'WAF', 'CAF', 'NEAF', 'SEAF', 'WSAF', 'ESAF', 'MDG', 'RAR', 'WSB',
                  'ESB', 'RFE', 'WCA', 'ECA', 'TIB', 'EAS', 'ARP', 'SAS', 'SEA', 'NAU',
                  'CAU', 'EAU', 'SAU', 'NZ']#, 'EAN', 'WAN']
nreg = len(rlabels)

ar6_land = regionmask.defined_regions.ar6.land

### Berkeley Earth

This data has nlat = 180, nlon = 360 so it is 1deg data

In [ ]:
berkdir = '/g/data/w97/azh561/cmip_heatwave/BerkleyEarth/'

# Define the filename convention:
bprefix = 'EHF_'
bendfix = '_summer_BerkleyEarth_1950-2014.nc'

# for the cumulative heat
hwcfile = 'EHF_HWC_summer_BerkleyEarth_1950-2014.nc'
ehifile = 'BerkleyEarth_HW_daily_EHI_value_1950_2017_bp_1950_2014.nc'
spellfile = 'BerkleyEarth_HW_daily_EHF_1950_2017_bp_1950_2014.nc'

### CMIP6 data

They are all on different resolutions! So will require regridding to plot and evaluate:

    - finest resolution: nlat = 256, nlon = 512 -> 0.7 deg x 0.7 deg
    - coarsest resolution: nlat = 96, nlon = 144 -> 1.875 deg x 2.5 deg

In [ ]:
cmip6dir = '/g/data/w97/azh561/cmip_heatwave/cmip6_ehf/'

# Define the models
models6 = ['ACCESS-CM2','AWI-CM-1-1-MR','AWI-ESM-1-1-LR','BCC-CSM2-MR','BCC-ESM1','CanESM5','CMCC-CM2-SR5',
'EC-Earth3-AerChem','EC-Earth3','EC-Earth3-Veg','EC-Earth3-Veg-LR','FGOALS-f3-L','FGOALS-g3','GFDL-CM4',
'GFDL-ESM4','GISS-E2-1-G','INM-CM4-8','INM-CM5-0','IPSL-CM6A-LR','KACE-1-0-G','KIOST-ESM','MIROC6',
'MPI-ESM-1-2-HAM','MPI-ESM1-2-HR','MPI-ESM1-2-LR','MRI-ESM2-0','NESM3','NorCPM1','NorESM2-LM','NorESM2-MM','TaiESM1']
nmod6 = len(models6)

# Define the filename convention:
mendfix6 = '_historical_r1i1p1f1_yearly_summer.nc'

# The EHI files
ehiefix6 = '_historical_r1i1p1f1_daily.nc'

### CMIP6 Models with 10 members

In [ ]:
mmdir = '/g/data/w97/azh561/cmip_heatwave/cmip6_sme_ehf/'
mmendfix = '_yearly_summer.nc'

mm1 = ['IPSL-CM6A-LR_historical_r2i1p1f1','IPSL-CM6A-LR_historical_r3i1p1f1','IPSL-CM6A-LR_historical_r4i1p1f1',
'IPSL-CM6A-LR_historical_r5i1p1f1','IPSL-CM6A-LR_historical_r6i1p1f1','IPSL-CM6A-LR_historical_r7i1p1f1',
'IPSL-CM6A-LR_historical_r8i1p1f1','IPSL-CM6A-LR_historical_r9i1p1f1','IPSL-CM6A-LR_historical_r10i1p1f1']

mm2 = ['MIROC6_historical_r2i1p1f1','MIROC6_historical_r3i1p1f1','MIROC6_historical_r4i1p1f1',
       'MIROC6_historical_r5i1p1f1','MIROC6_historical_r6i1p1f1','MIROC6_historical_r7i1p1f1',
       'MIROC6_historical_r8i1p1f1','MIROC6_historical_r9i1p1f1','MIROC6_historical_r10i1p1f1']

mm3 = ['MPI-ESM1-2-HR_historical_r2i1p1f1','MPI-ESM1-2-HR_historical_r3i1p1f1','MPI-ESM1-2-HR_historical_r4i1p1f1',
'MPI-ESM1-2-HR_historical_r5i1p1f1','MPI-ESM1-2-HR_historical_r6i1p1f1','MPI-ESM1-2-HR_historical_r7i1p1f1',
'MPI-ESM1-2-HR_historical_r8i1p1f1','MPI-ESM1-2-HR_historical_r9i1p1f1','MPI-ESM1-2-HR_historical_r10i1p1f1']

mm4 = ['CanESM5_historical_r2i1p1f1','CanESM5_historical_r3i1p1f1','CanESM5_historical_r4i1p1f1',
      'CanESM5_historical_r5i1p1f1','CanESM5_historical_r6i1p1f1','CanESM5_historical_r7i1p1f1',
      'CanESM5_historical_r8i1p1f1','CanESM5_historical_r9i1p1f1','CanESM5_historical_r10i1p1f1']

mm5 = ['UKESM1-0-LL_historical_r1i1p1f2','UKESM1-0-LL_historical_r2i1p1f2','UKESM1-0-LL_historical_r3i1p1f2',
'UKESM1-0-LL_historical_r4i1p1f2','UKESM1-0-LL_historical_r8i1p1f2','UKESM1-0-LL_historical_r9i1p1f2']

### CMIP5 MME

In [ ]:
cmip5dir = '/g/data/w97/azh561/cmip_heatwave/cmip5_ehf/'

# Define the models
models5 = ['ACCESS1-0','ACCESS1.3','bcc-csm1-1','bcc-csm1-1-m','BNU-ESM',
          'CanESM2','CCSM4','CESM1-BGC','CESM1-CAM5','CMCC-CESM',
'CMCC-CM','CMCC-CMS','CNRM-CM5','CSIRO-Mk3-6-0','FGOALS_g2','GFDL-CM3',
'GFDL-ESM2G','GFDL-ESM2M',#'HadGEM2-AO','HadGEM2-CC','HadGEM2-ES',
          'inmcm4',
'IPSL-CM5A-LR','IPSL-CM5A-MR','IPSL-CM5B-LR','MIROC5','MIROC-ESM-CHEM',
'MIROC-ESM','MPI-ESM-LR','MPI-ESM-MR','MRI-CGCM3','MRI-ESM1','NorESM1-M']

nmod5 = len(models5)

# Define the filename convention:
mendfix5 = '_historical_r1i1p1_yearly_summer.nc'

# The EHI files
ehiefix5 = '_historical_r1i1p1_daily.nc'


### Define some functions

In [ ]:
# If longitudes are 0 to 360 need to shift to -180 to 180
# https://stackoverflow.com/questions/53345442/about-changing-longitude-array-from-0-360-to-180-to-180-with-python-xarray
def fix_lon(dm,lon_name):
    '''Function to adjust longitudes'''
    dm['_longitude_adjusted'] = xr.where(dm[lon_name] > 180,dm[lon_name] - 360,dm[lon_name])
    # reassign the new coords to as the main lon coords and sort DataArray using new coordinate values
    dm = (
        dm
        .swap_dims({lon_name: '_longitude_adjusted'})
        .sel(**{'_longitude_adjusted': sorted(dm._longitude_adjusted)})
        .drop(lon_name))
    dm = dm.rename({'_longitude_adjusted': lon_name})
    
    return dm

In [ ]:
def hwseason(data,spell,yr,variable):
    
    # Define the output array
    dataout = np.empty((len(data['lat'].values),len(data['lon'].values)),dtype=np.float64)

    # Select relevant data, mask for non-HW days and sum across the Northern Hemisphere MJJAS
    nhtmp = data[variable].loc['%s-05-01' %(yr):'%s-09-30' %(yr),:,:]
    nhspell = spell['event'].loc['%s-05-01' %(yr):'%s-09-30' %(yr),:,:]
    nhma = np.ma.masked_array(nhtmp,nhspell<=0).filled(np.nan)

    dataout[90:,:] = np.nansum(nhma[:,90:,:],axis=0)
    del nhtmp,nhspell,nhma
    
    # Select relevant data, mask for non-HW days and sum across the Southern Hemisphere NDJFM
    shtmp1 = data[variable].loc['%s-11-01' %(yr):'%s-12-31' %(yr),:,:]
    shspell1 = spell['event'].loc['%s-11-01' %(yr):'%s-12-31' %(yr),:,:]
    shma1 = np.ma.masked_array(shtmp1,shspell1<=0).filled(np.nan)

    if yr == 2014:
        dataout[:90,:] = np.nansum(shma1[:,:90,:],axis=0)
    else:
        shtmp2 = data[variable].loc['%s-01-01' %(yr+1):'%s-03-31' %(yr+1),:,:]
        shspell2 = spell['event'].loc['%s-01-01' %(yr+1):'%s-03-31' %(yr+1),:,:]
        shma2 = np.ma.masked_array(shtmp2,shspell2<=0).filled(np.nan)
        dataout[:90,:] = np.nansum(shma1[:,:90,:],axis=0) + np.nansum(shma2[:,:90,:],axis=0)
        del shtmp2,shspell2,shma2
    del shtmp1,shspell1,shma1
    
    return dataout

In [ ]:
def process_model(hwind,datadir,prefix,modellist,endfix):
    
    nmod = len(modellist)
    
    moddata = np.empty((nmod,nyrs,nlat,nlon),dtype=np.float64)
    
    for mind in range(nmod):
        dm = xr.open_dataset('%s%s%s%s' %(datadir,prefix,modellist[mind],endfix),decode_times=False)
        
        if np.nanmax(dm['lon']) > 200.:
            dm = fix_lon(dm,'lon')

        if dm[hwind].shape[0] > nyrs:
            dm = dm.isel(time=slice(100,165))
            
        # regrid data
        datavalue = dm[hwind]
        regridder = xe.Regridder(dm, ds_out, 'bilinear')
        moddata[mind,:,:,:] = regridder(datavalue)
        del dm, datavalue,regridder
            
    return moddata

In [ ]:
def process_model_hwc(hwind,datadir,prefix1,prefix2,modellist,endfix):
    
    nmod = len(modellist)
    
    moddata = np.empty((nmod,nyrs,nlat,nlon),dtype=np.float64)

    for mind in range(nmod):
        
        # Open data
        dm = xr.open_dataset('%s%s%s%s' %(datadir,prefix1,modellist[mind],endfix))
        dms = xr.open_dataset('%s%s%s%s' %(datadir,prefix2,modellist[mind],endfix))
        
        # Fix longitudes if required
        if np.nanmax(dm['lon']) > 200.:                
            dm = fix_lon(dm,'lon')
        if np.nanmax(dms['lon']) > 200.:
            dms = fix_lon(dms,'lon')

        # calculate HWD
        datahwseason = np.empty((nyrs,len(dm['lat'].values),len(dm['lon'].values)),dtype=np.float64)
        yr=1950
        for yy in range(nyrs):
            datahwseason[yy,:,:] = hwseason(dm,dms,yr,'EHIsig')
            yr += 1

        dm_new = xr.Dataset({
            'HWC': xr.DataArray(
                    data   = datahwseason,
                    dims   = ['year','lat','lon'],
                    coords = {'year': np.arange(1950,2015),'lat': dm['lat'].values,'lon': dm['lon'].values},
                    )
                })
        
        dm_new.to_netcdf('/g/data/w97/azh561/cmip_heatwave/HWC/HWC_%s.nc' %(modellist[mind]))
        
        # Regrid data 
        datavalue = dm_new['HWC']
        regridder = xe.Regridder(dm_new, ds_out, 'bilinear')
        moddata[mind,:,:,:] = regridder(datavalue)
        del dm,dms,dm_new,datavalue,regridder,datahwseason
        
    moddata[:,-1,:,:] = np.nan
    
    return moddata

In [ ]:
process_model_hwc('HWC',mmdir,'EHI_heatwaves_','EHF_heatwaves_',mm1,'_daily.nc')
process_model_hwc('HWC',mmdir,'EHI_heatwaves_','EHF_heatwaves_',mm2,'_daily.nc')
process_model_hwc('HWC',mmdir,'EHI_heatwaves_','EHF_heatwaves_',mm3,'_daily.nc')
process_model_hwc('HWC',mmdir,'EHI_heatwaves_','EHF_heatwaves_',mm4,'_daily.nc')
process_model_hwc('HWC',mmdir,'EHI_heatwaves_','EHF_heatwaves_',mm5,'_daily.nc')

In [ ]:
# Function to plot data
def plot_kde(bedata,cmip6,cmip5,mm1,mm2,mm3,mm4,mm5,vlabel,rlabel,xlim,ylim,figurename):
    """This function plots the kde """
          
    # Create figure object and subplots
    fig, ax = plt.subplots(figsize=(5,5))

    plt.rcParams['savefig.dpi']=300
    plt.rcParams["font.weight"] = "bold"
    plt.rcParams["axes.labelweight"] = "bold"
    plt.rcParams["axes.titleweight"] = "bold"
    plt.rcParams["axes.titlepad"] = 5.0

    # Plot CMIP6
    sns.kdeplot(np.reshape(cmip6,-1), ax=ax, shade=False, color='red', linestyle='-',linewidth=3,label='CMIP6') # All years

    # Plot CMIP5
    sns.kdeplot(np.reshape(cmip5,-1), ax=ax, shade=False, color='blue', linestyle='-',linewidth=3,label='CMIP5') # All years

    # Plot Individual Model Ensembles
    
    evenly_spaced_interval = np.linspace(0.2, 0.8, 5)
    mycolors = [plt.cm.binary(x) for x in evenly_spaced_interval]
   
    sns.kdeplot(np.reshape(mm1,-1), ax=ax, shade=False, color=mycolors[0], linestyle='-',linewidth=1,label='IPSL') # All years
    sns.kdeplot(np.reshape(mm2,-1), ax=ax, shade=False, color=mycolors[1], linestyle='-',linewidth=1,label='MIROC') # All years
    sns.kdeplot(np.reshape(mm3,-1), ax=ax, shade=False, color=mycolors[2], linestyle='-',linewidth=1,label='MPI-ESM') # All years
    sns.kdeplot(np.reshape(mm4,-1), ax=ax, shade=False, color=mycolors[3], linestyle='-',linewidth=1,label='CanESM') # All years
    sns.kdeplot(np.reshape(mm5,-1), ax=ax, shade=False, color=mycolors[4], linestyle='-',linewidth=1,label='UKESM') # All years

    # Plot Berkeley Earth
    sns.kdeplot(np.reshape(bedata,-1), ax=ax, shade=False, color='black', linestyle='-',linewidth=3,label='BE') # All years

    # Labelling of subplot panels
    ax.set_xlabel('%s' %(vlabel), fontweight = 'bold')
    ax.set_ylabel("Density Function", fontweight = 'bold')

    # Title
    ax.set_title('%s' %(rlabel), fontweight = 'bold')

    # Set common axis limits
    ax.set_ylim(0,ylim)
    ax.set_xlim(0,xlim)

    # Add legend
    plt.legend()
    
    fig.savefig(figurename, transparent=True)
    plt.close(fig)

In [ ]:
def kde_reg_wgt_avg(indata,st,fi):

    # region mask
    maskreg = ar6_land.mask(ds_out)
    
    if len(indata.shape) == 3:
    
        nyrs,nlat,nlon = indata.shape
        mask = np.repeat(maskreg.values[np.newaxis,...],nyrs,axis=0)

    else:

        nmod,nyrs,nlat,nlon = indata.shape
        masktmp = np.repeat(maskreg.values[np.newaxis,...],nyrs,axis=0)
        mask = np.repeat(masktmp[np.newaxis,...],nmod,axis=0)

    regdata = np.ma.masked_array(indata,(mask<st)|(mask>fi)).filled(np.nan)

    # calculate the area-weighted average
    latr = np.deg2rad(lat2d)
    weights = np.cos(latr)

    # Loop through the years
    
    if len(indata.shape) == 3:
        datawgt = np.empty((nyrs),dtype=np.float64)
        for yy in range(nyrs):
            datawgt[yy] = np.ma.average(np.ma.MaskedArray(regdata[yy,:,:], mask=np.isnan(regdata[yy,:,:])),weights=weights) 
    else:
        datawgt = np.empty((nmod,nyrs),dtype=np.float64)

        for yy in range(nyrs):
            for mm in range(nmod):
                datawgt[mm,yy] = np.ma.average(np.ma.MaskedArray(regdata[mm,yy,:,:], mask=np.isnan(regdata[mm,yy,:,:])),weights=weights)

    return datawgt

### Loop through the heatwave indices 

In [ ]:
# Define the index, label and axis limit
var = ['HWF','HWD','HWM','HWC']
nvar = len(var)
modvar = ['HWF_EHF','HWD_EHF','HWM_EHF','HWC_EHF']
vlabels = ['HWF [days]','HWD [days]','HWM [\xb0 $C^{2}$]','HWC [\xb0 C]']
vlim = [50,20,20,50]
vxlim = [0.20,0.5,0.5,0.20]

# Define dimensions
nyrs = 65
nlat = 180
nlon = 360

regst = [1,9,16,20,28,31,39]
regfi = [6,15,19,27,30,37,42]
regnm = ["North America","South America","Europe","Africa","Russia","Asia","Australia"]

In [ ]:
for eind in range(nvar):
    
    print('Calculating and plotting %s' %(var[eind]))
    
    # Read in previously calculated HWC
    if var[eind] in ['HWC']:
        
        # Read in the Berkeley Earth data 
        berk = xr.open_dataset('/g/data/w97/azh561/cmip_heatwave/HWC/HWC_Berkeley_Earth.nc')
        berkma = np.ma.masked_array(berk[var[eind]].values,berk[var[eind]].values<=0).filled(np.nan)
        berkma[-1,:,:] = np.nan # Make the last year nan as model data for last summer season in the Southern Hemisphere is incomplete
        ds_out = xr.Dataset({'lat': (['lat'], berk['lat'].values),'lon': (['lon'], berk['lon'].values),})
        
        # CMIP6 models
        moddata6 = process_model('HWC','/g/data/w97/azh561/cmip_heatwave/HWC/cmip6/','EHI_heatwaves_',models6,'_historical_r1i1p1f1_HWC.nc')
        moddata6[:,-1,:,:] = np.nan
        
        # CMIP5 models
        moddata5 = process_model('HWC','/g/data/w97/azh561/cmip_heatwave/HWC/cmip5/','EHI_heatwaves_',models5,'_historical_r1i1p1_HWC.nc')
        moddata5[:,-1,:,:] = np.nan
        
        # Individual Model Ensembles
        mm1data = process_model('HWC','/g/data/w97/azh561/cmip_heatwave/HWC/','HWC_',mm1,'.nc')
        mm1data[:,-1,:,:] = np.nan
        mm2data = process_model('HWC','/g/data/w97/azh561/cmip_heatwave/HWC/','HWC_',mm2,'.nc')
        mm2data[:,-1,:,:] = np.nan
        mm3data = process_model('HWC','/g/data/w97/azh561/cmip_heatwave/HWC/','HWC_',mm3,'.nc')
        mm3data[:,-1,:,:] = np.nan
        mm4data = process_model('HWC','/g/data/w97/azh561/cmip_heatwave/HWC/','HWC_',mm4,'.nc')
        mm4data[:,-1,:,:] = np.nan
        mm5data = process_model('HWC','/g/data/w97/azh561/cmip_heatwave/HWC/','HWC_',mm5,'.nc')
        mm5data[:,-1,:,:] = np.nan

    else:
        
        # Read in the Berkeley Earth data - decode times necessary as HWF and HWD in units of days
        berk = xr.open_dataset('%s%s%s%s' %(berkdir,bprefix,var[eind],bendfix),decode_times=False)
        berkma = np.ma.masked_array(berk[var[eind]].values,berk[var[eind]].values<=0).filled(np.nan)

        # Define the common grid for the models to regrid too!
        ds_out = xr.Dataset({'lat': (['lat'], berk['latitude'].values),'lon': (['lon'], berk['longitude'].values),})

        # CMIP6 models
        moddata6 = process_model(modvar[eind],cmip6dir,'EHF_heatwaves_',models6,mendfix6)
        
        # CMIP5 models
        moddata5 = process_model(modvar[eind],cmip5dir,'EHF_heatwaves_',models5,mendfix5)

        # Individual Model Ensembles
        mm1data = process_model(modvar[eind],mmdir,'EHF_heatwaves_',mm1,mmendfix)
        mm2data = process_model(modvar[eind],mmdir,'EHF_heatwaves_',mm2,mmendfix)
        mm3data = process_model(modvar[eind],mmdir,'EHF_heatwaves_',mm3,mmendfix)
        mm4data = process_model(modvar[eind],mmdir,'EHF_heatwaves_',mm4,mmendfix)
        mm5data = process_model(modvar[eind],mmdir,'EHF_heatwaves_',mm5,mmendfix)
        
    _, lat2d = np.meshgrid(ds_out['lon'].values, ds_out['lat'].values)

    # Define the land-sea mask (region = 0, elsewhere is nan)
    land_110 = regionmask.defined_regions.natural_earth.land_110
    land_mask = land_110.mask(ds_out)
    mask2d = ar6_land.mask(ds_out) * land_mask.squeeze(drop=True)

    # Define the land-sea mask (region = 0, elsewhere is nan)
    maskreg = ar6_land.mask(ds_out)
    maskland = maskreg * land_mask.squeeze(drop=True)
    maskland3d = np.repeat(maskland.values[np.newaxis,...],nyrs,axis=0)
        
    # Mask the missing values as these have not been set to nan in the files
    modma6 = np.ma.masked_array(moddata6,moddata6<=0).filled(np.nan)
    modma5 = np.ma.masked_array(moddata5,moddata5<=0).filled(np.nan)
    mm1ma5 = np.ma.masked_array(mm1data,mm1data<=0).filled(np.nan)
    mm2ma5 = np.ma.masked_array(mm2data,mm2data<=0).filled(np.nan)
    mm3ma5 = np.ma.masked_array(mm3data,mm3data<=0).filled(np.nan)
    mm4ma5 = np.ma.masked_array(mm4data,mm4data<=0).filled(np.nan)
    mm5ma5 = np.ma.masked_array(mm5data,mm5data<=0).filled(np.nan)
           
    # Mask ocean points
    berklm = np.ma.masked_array(berkma,maskland3d!=0).filled(np.nan)
    modlm6 = np.ma.masked_array(modma6,np.repeat(maskland3d[np.newaxis,...],nmod6,axis=0)!=0).filled(np.nan)
    modlm5 = np.ma.masked_array(modma5,np.repeat(maskland3d[np.newaxis,...],nmod5,axis=0)!=0).filled(np.nan)
    mm1lm5 = np.ma.masked_array(mm1ma5,np.repeat(maskland3d[np.newaxis,...],len(mm1),axis=0)!=0).filled(np.nan)
    mm2lm5 = np.ma.masked_array(mm2ma5,np.repeat(maskland3d[np.newaxis,...],len(mm2),axis=0)!=0).filled(np.nan)
    mm3lm5 = np.ma.masked_array(mm3ma5,np.repeat(maskland3d[np.newaxis,...],len(mm3),axis=0)!=0).filled(np.nan)
    mm4lm5 = np.ma.masked_array(mm4ma5,np.repeat(maskland3d[np.newaxis,...],len(mm4),axis=0)!=0).filled(np.nan)
    mm5lm5 = np.ma.masked_array(mm5ma5,np.repeat(maskland3d[np.newaxis,...],len(mm5),axis=0)!=0).filled(np.nan)
    
    # Plot the KDEs for each region
    for rr in range(len(regnm)):
        berkwgt = kde_reg_wgt_avg(berklm,regst[rr],regfi[rr])
        mwgt6 = kde_reg_wgt_avg(modlm6,regst[rr],regfi[rr])
        mwgt5 = kde_reg_wgt_avg(modlm5,regst[rr],regfi[rr])
        m1wgt5 = kde_reg_wgt_avg(mm1lm5,regst[rr],regfi[rr])
        m2wgt5 = kde_reg_wgt_avg(mm2lm5,regst[rr],regfi[rr])
        m3wgt5 = kde_reg_wgt_avg(mm3lm5,regst[rr],regfi[rr])
        m4wgt5 = kde_reg_wgt_avg(mm4lm5,regst[rr],regfi[rr])
        m5wgt5 = kde_reg_wgt_avg(mm5lm5,regst[rr],regfi[rr])
        fname = "%s_%s.png" %(var[eind],regnm[rr])
        plot_kde(berkwgt,mwgt6,mwgt5,m1wgt5,m2wgt5,m3wgt5,m4wgt5,m5wgt5,vlabels[eind],regnm[rr],vlim[eind],vxlim[eind],fname)
        del berkwgt,mwgt6,mwgt5,m1wgt5,m2wgt5,m3wgt5,m4wgt5,m5wgt5
 
    # Clean up
    del berkma,berklm,moddata6,modma6,modlm6,moddata5,modma5,modlm5
    del mm1data,mm1ma5,mm1lm5
    del mm2data,mm2ma5,mm2lm5
    del mm3data,mm3ma5,mm3lm5
    del mm4data,mm4ma5,mm4lm5
    del mm5data,mm5ma5,mm5lm5
    